In [1]:
from zipfile import ZipFile
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
def gen_parcer(f):
    for line in f.readlines():
        yield line.decode().strip().split("\t")

In [3]:
with ZipFile('data.zip') as datazip:
    with datazip.open('train.tsv') as f:
        pd_train = pd.DataFrame(gen_parcer(f),
                                columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply",
                                         "label", "confidence"])
    with datazip.open('public.tsv') as f:
         pd_public = pd.DataFrame(gen_parcer(f),
                                  columns=["context_id", "context_2", "context_1", "context_0", "reply_id", "reply"])

In [4]:
pd_train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [5]:
pd_public.head()

,context_id,context_2,context_1,context_0,reply_id,reply
0,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,0,неа .
1,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,1,"нет , не хочу ."
2,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,2,нет .
3,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,3,"конечно , нет ."
4,138920940977,"знаешь , я иногда подумываю , что тебе надо пр...",не - а .,нет ?,4,"разумеется , нет ."


In [6]:
pd_train.count()

context_id    97533
context_2     97533
context_1     97533
context_0     97533
reply_id      97533
reply         97533
label         97533
confidence    97533
dtype: int64

In [7]:
pd_public.count()

context_id    9968
context_2     9968
context_1     9968
context_0     9968
reply_id      9968
reply         9968
dtype: int64

In [8]:
target_encoder = {'bad': 0, 'good': 2, 'neutral': 1}  
pd_train.label = pd_train.label.map(target_encoder)

In [9]:
pd_train.confidence = pd_train.confidence.astype(float)

In [10]:
pd_data = pd.concat([pd_train, pd_public], axis=0).reset_index(drop=True)
pd_data.label.fillna(-1, inplace=True)
pd_data.confidence.fillna(0.0, inplace=True)

# Что за данные

In [11]:
pd_train.shape, pd_public.shape, pd_data.shape

((97533, 8), (9968, 6), (107501, 8))

In [12]:
pd_data.groupby("label")["context_id"].count() / len(pd_data)

label
-1.0    0.092725
 0.0    0.323439
 1.0    0.104687
 2.0    0.479149
Name: context_id, dtype: float64

# Простой бейзлайн

## features

In [13]:
from scipy import sparse

In [14]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
cv_all = CountVectorizer()
s_words = pd_data[["context_2", "context_1", "context_0", "reply"]].apply(lambda x: " ".join(x), axis=1)
cv_all.fit(s_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
tfidf_all = TfidfVectorizer()

In [17]:
def fit_tfidf(**kwargs):
    global tfidf_all
    tfidf_all = TfidfVectorizer(**kwargs)
    s_words = pd_data[["context_2", "context_1", "context_0", "reply"]].apply(lambda x: " ".join(x), axis=1)
    tfidf_all.fit(s_words)

In [18]:
def to_features_cv(df):
    context = cv_all.transform(df[["context_2", "context_1", "context_0"]].apply(lambda x: " ".join(x), axis=1))
    reply = cv_all.transform(df["reply"])
    features = sparse.hstack([context, reply])
    return features

In [19]:
def to_features_tfidf(df):
    context = tfidf_all.transform(df[["context_2", "context_1", "context_0"]].apply(lambda x: " ".join(x), axis=1))
    reply = tfidf_all.transform(df["reply"])
    features = sparse.hstack([context, reply])
    return features

## vectors

- rusvectores
- nlpub (russion distribution t...)

## crossval

In [20]:
from tqdm import tqdm_notebook

In [21]:
from sklearn.model_selection import StratifiedKFold

In [22]:
skf = StratifiedKFold(n_splits=3)

In [23]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor

In [24]:
class RandomMinMaxRegressor(object):
    def __init__(self):
        self.min_y = None
        self.max_y = None
    
    def fit(self, x, y):
        self.min_y = y.min()
        self.max_y = y.max()
    
    def predict(self, x):
        return np.random.random(x.shape[0]) * (self.max_y - self.min_y) + self.min_y        

In [25]:
from sklearn.metrics import mean_absolute_error

In [26]:
def calc_ndcg_score(df, y_predict):
    df = df.copy()
    df["predict"] = y_predict
    #dcg
    df.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["dcg"] = df["label"] / np.log2(df["row_count"] + 1)
    #idcg
    df.sort_values(["context_id", "label"], ascending=[True, False], inplace=True)
    df["row_count"] = df.groupby("context_id").cumcount()+1
    df["idcg"] = df["label"] / np.log2(df["row_count"] + 1)
    
    df_context = df.groupby("context_id").sum()
    return (df_context["dcg"] / df_context["idcg"]).mean() * 100000

In [71]:
def do_cross_val(model_reg, p_sampling, to_features, to_target, sampling_params):
    mae_test = []
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_train = p_sampling(pd_train.iloc[_train], **sampling_params)
        i_test = pd_train.iloc[_test]

        X_train = to_features(i_train)
        X_test = to_features(i_test)
        Y_train = to_target(i_train)
        Y_test = to_target(i_test)

        model_reg.fit(X_train, Y_train)
        Y_predict = model_reg.predict(X_test)
        
        mae_test.append(mean_absolute_error(Y_train, model_reg.predict(X_train)))
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(sorted(mae_test))
    print(sorted(mae_errors))
    print(sorted(ndcg_scores))

## target

In [28]:
trg_label = lambda x: x.label.values

In [29]:
trg_label_confidence = lambda x: (x.label.values - 1) * x.confidence.values.astype(np.float)

## Нормальные варианты

In [30]:
from sklearn.linear_model import Ridge

In [31]:
fit_tfidf(analyzer='char', ngram_range=(3,4))

In [72]:
do_cross_val(Ridge(alpha=20.0), m_sampling, to_features_tfidf, trg_label,
             {})

A Jupyter Widget

Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




[0.7707396527255045, 0.771191720398295, 0.7714803030187314]
[0.8226197062858732, 0.8234261880833347, 0.8234458046155703]
[86250.81379658228, 86463.30007075329, 86613.92412908383]


In [80]:
do_cross_val(Ridge(alpha=20.0), m_sampling, to_features_tfidf, trg_label,
             {})

A Jupyter Widget


[0.9447154995288046, 0.9448030371268662, 0.9449557366445199]
[0.8412207827569289, 0.8425614524114927, 0.8429086689676474]
[85531.86049534313, 85576.6779517952, 85803.07900825172]


In [81]:
import lightgbm

In [82]:
do_cross_val(lightgbm.LGBMRegressor(), m_sampling, to_features_tfidf, trg_label,
             {})

A Jupyter Widget

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')



[0.9473249493145631, 0.9478841272710248, 0.9479454209874395]
[0.8400871509948474, 0.841501459212799, 0.8416204327962735]
[85461.05223789712, 85468.76426682134, 85680.45174335346]


In [73]:
do_cross_val(Ridge(alpha=20.0), m_sampling, to_features_tfidf, trg_label,
             {"p_con0_mul": 1})

A Jupyter Widget


[0.756170131256931, 0.7565562164899753, 0.7569632328059338]
[0.819359380002962, 0.8196217519678458, 0.819749389971439]
[86283.3338542383, 86401.44072599198, 86572.15282855151]


In [74]:
do_cross_val(Ridge(alpha=20.0), m_sampling, to_features_tfidf, trg_label,
             {"p_plus": 0.20})

A Jupyter Widget


[0.7975784229318101, 0.7987264967410189, 0.79989051592151]
[0.8363488366512923, 0.837426607204882, 0.8374353175381992]
[86240.32439458236, 86252.92109078343, 86548.8202303653]


In [77]:
do_cross_val(Ridge(alpha=1.0), m_sampling, to_features_tfidf, trg_label,
             {"p_plus": .10, "p_con0_mul": 1})

A Jupyter Widget


[0.6024565308531346, 0.6026688657792648, 0.6027879598479483]
[0.8202469613339363, 0.8213296777318918, 0.8220450205647197]
[85695.66577442182, 85706.08971794875, 85826.71108508714]


In [78]:
pd_os_good =pd.read_pickle("pd_os_good.pickle")
pd_os_bad = pd.read_pickle("pd_os_bad.pickle")

In [79]:
def m_sampling(df, remove=None, p_plus=None, p_con0_mul=None):
    size = len(df)
    
    if remove is not None:
        df = df.sort_values("confidence").iloc[int(size * remove):]
        
    if p_plus is not None:
        ix_context = np.random.choice(df.context_id.unique(), size=int(size * p_plus))
        df_app = pd_train[pd_train.context_id.isin(ix_context)]
        df_app = df_app[~df_app.duplicated(["context_id"], keep="first")].copy()
        df_app["reply"] = np.random.choice(df[df.label.eq(0)].reply, size=len(df_app))
        df_app["label"] = 0
        df_app["confidence"] = 0.7
        df = pd.concat([df, df_app]).reset_index(drop=True)
    
    if p_con0_mul is not None:
        df_app = df[~df.context_2.eq("")][["context_id", "context_2", "context_1"]]
        df_app = df_app[~df_app.duplicated(["context_id"], keep="first")].copy()
        df_app["context_id"] = 0
        df_app.rename(columns={"context_2": "context_0", "context_1": "reply"}, inplace=True)
        df_app["context_2"] = ""
        df_app["context_1"] = ""
        df_app["label"] = 2
        df_app["confidence"] = 0.7
        df = pd.concat([df, df_app]).reset_index(drop=True)
        
    df = pd.concat([df, pd_os_good, pd_os_bad]).reset_index(drop=True)
        
    return df.copy()

In [65]:
m_sampling(pd_train, p_plus=0.9, p_con0_mul=1)

,confidence,context_0,context_1,context_2,context_id,label,reply,reply_id
0,0.875352,"ладно , повесь трубку .","о , я тебя вижу .","кликни на меня а потом на надпись "" видео - зв...",22579918886,2,не могу .,0
1,0.900968,"ладно , повесь трубку .","о , я тебя вижу .","кликни на меня а потом на надпись "" видео - зв...",22579918886,1,"нет , звонить буду я .",1
2,0.884320,"ладно , повесь трубку .","о , я тебя вижу .","кликни на меня а потом на надпись "" видео - зв...",22579918886,0,"слушай , я не мог уйти .",2
3,0.982530,"ладно , повесь трубку .","о , я тебя вижу .","кликни на меня а потом на надпись "" видео - зв...",22579918886,2,я не прекращу звонить .,3
4,0.838054,"ладно , повесь трубку .","о , я тебя вижу .","кликни на меня а потом на надпись "" видео - зв...",22579918886,2,я звоню им .,4
5,0.955718,"ладно , повесь трубку .","о , я тебя вижу .","кликни на меня а потом на надпись "" видео - зв...",22579918886,0,просто повесь трубку .,5
6,0.909115,"я пытаюсь добраться до юджина , но , кажется ,...","слушайте , мы с женой . . . совсем заблудились .",бывало и получше .,50117448291,0,едете ?,0
7,0.984621,"я пытаюсь добраться до юджина , но , кажется ,...","слушайте , мы с женой . . . совсем заблудились .",бывало и получше .,50117448291,0,едем .,1
8,0.833307,"я пытаюсь добраться до юджина , но , кажется ,...","слушайте , мы с женой . . . совсем заблудились .",бывало и получше .,50117448291,2,"куда же вы едете , месье ?",2
9,0.800272,"я пытаюсь добраться до юджина , но , кажется ,...","слушайте , мы с женой . . . совсем заблудились .",бывало и получше .,50117448291,2,"берите бутылку , а мы такси .",3


In [38]:
df = pd_train

In [59]:
df_app = df[~df.context_2.eq("")][["context_id", "context_2", "context_1"]]
df_app = df_app[~df_app.duplicated(["context_id"], keep="first")].copy()

In [63]:
df.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,2,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",1,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",0,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,2,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,2,0.838054


In [62]:
df_app["context_id"] = 0
df_app.rename(columns={"context_2": "context_0", "context_1": "reply"}, inplace=True)
df_app["context_2"] = ""
df_app["context_1"] = ""
df_app["label"] = 2
df_app["confidence"] = 0.7

,context_id,context_2,context_1
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу ."
6,50117448291,бывало и получше .,"слушайте , мы с женой . . . совсем заблудились ."
12,72855813972,"ну , это не ново .","кажется , я там ни разу не был ."
18,76323501777,ага .,"он был хорошим псом , линдси ?"
24,127768564286,"как вы предпочитаете "" похищенные "" или "" подо...","я предпочитаю ни то , ни другое , но мой напар..."
30,135053376199,"как я уже говорил , тут 4 спальни .","я понимаю , вы это ожидали ."
31,213012655515,не хочу иметь дела с негативными последствиями .,надеюсь вы больше не собираетесь сегодня меня ...
37,243724867653,ответь мне .,оуэн .
43,275211951503,и вы стоите миллионы долларов городу ущерба .,я всегда хотела сделать это .
55,297851249865,я позвоню федералам .,они тебе ничего не скажут .


## Неудачные варианты

In [37]:
fit_tfidf(max_features=10000)
do_cross_val(Lasso(alpha=0.01), to_features_tfidf, trg_label_confidence)

A Jupyter Widget


[0.75838986817906762, 0.75749894796205131, 0.75695730065427458]
[82749.59984256358, 82818.22569952125, 82951.3881202625]


In [60]:
from sklearn.linear_model import ElasticNet
fit_tfidf(max_features=10000)
do_cross_val(ElasticNet(alpha=1.0), to_features_tfidf, trg_label_confidence)

A Jupyter Widget


[0.75838986817906762, 0.75749894796205131, 0.75695730065427458]
[82749.59984256358, 82818.22569952125, 82951.3881202625]


In [35]:
fit_tfidf(max_features=20000)
do_cross_val(LinearRegression(), to_features_tfidf)

A Jupyter Widget


[1.1641773562812208, 1.2216055918557263, 1.2445674321405]
[85160.73563781707, 85201.6761086984, 85174.57705347847]


In [36]:
fit_tfidf(max_features=5000)
do_cross_val(LinearRegression(), to_features_tfidf)

A Jupyter Widget


[1.1641773562812208, 1.2216055918557263, 1.2445674321405]
[85160.73563781707, 85201.6761086984, 85174.57705347847]


In [37]:
fit_tfidf(max_features=1000)
do_cross_val(LinearRegression(), to_features_tfidf)

A Jupyter Widget


[1.1641773562812208, 1.2216055918557263, 1.2445674321405]
[85160.73563781707, 85201.6761086984, 85174.57705347847]


In [38]:
fit_tfidf(max_features=200)
do_cross_val(LinearRegression(), to_features_tfidf)

A Jupyter Widget


[1.1641773562812208, 1.2216055918557263, 1.2445674321405]
[85160.73563781707, 85201.6761086984, 85174.57705347847]


In [38]:
do_cross_val(RandomMinMaxRegressor())

A Jupyter Widget


[0.94215100562127641, 0.94450351137377053, 0.94044066978111984]
[82635.87988159526, 82487.80155424896, 82826.7969582632]


In [49]:
def do_worst_ndcg():
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_test = pd_train.iloc[_test].copy().sort_values(["context_id", "label"], ascending=[True, False])
        
        Y_test = i_test.sort_values(["context_id", "label"], ascending=[True, False]).label.values
        Y_predict = i_test.sort_values(["context_id", "label"], ascending=[True, True]).label.values
        
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)
do_worst_ndcg()

A Jupyter Widget


[1.1195250984251968, 1.1126695579957553, 1.1291294986158105]
[77475.71260547417, 77562.26032466823, 77472.45724727474]


In [48]:
def do_best_ndcg():
    mae_errors = []
    ndcg_scores = []

    for _train, _test in tqdm_notebook(skf.split(pd_train.index, pd_train.label), total=3):
        i_test = pd_train.iloc[_test].copy().sort_values(["context_id", "label"], ascending=[True, False])
        
        Y_test = i_test.sort_values(["context_id", "label"], ascending=[True, False]).label.values
        Y_predict = i_test.sort_values(["context_id", "label"], ascending=[True, False]).label.values
        
        mae_errors.append(mean_absolute_error(Y_test, Y_predict))
        ndcg_scores.append(calc_ndcg_score(i_test, Y_predict))

    print(mae_errors)
    print(ndcg_scores)
do_best_ndcg()

A Jupyter Widget


[0.0, 0.0, 0.0]
[100000.0, 100000.0, 100000.0]


# Submit

In [81]:
def do_submit(submission_name, model_reg, to_features, to_target):
    i_train = pd_train
    i_test = pd_public.copy()

    X_train = to_features(i_train)
    X_test = to_features(i_test)
    Y_train = to_target(i_train)

    model_reg.fit(X_train, Y_train)
    Y_predict = model_reg.predict(X_test)

    i_test["predict"] = Y_predict
    i_test.sort_values(["context_id", "predict"], ascending=[True, False], inplace=True)

    i_test.head(15)
    i_test[["context_id", "reply_id"]].to_csv(submission_name, index=False, header=False, sep="\t")
    
    return i_test

In [82]:
fit_tfidf(analyzer="char", ngram_range=(3, 4))
do_submit("linear_cv_opsion.tsv", Ridge(alpha=20.0), to_features_tfidf, trg_label_confidence)

,context_id,context_2,context_1,context_0,reply_id,reply,predict
3479,100097508986637,,,выключай их,4,"пытаюсь , сэр .",0.345558
3478,100097508986637,,,выключай их,3,я пытаюсь !,0.162357
3476,100097508986637,,,выключай их,1,"ты не выключишь , тогда € сам выключу",-0.021206
3480,100097508986637,,,выключай их,5,выключите,-0.270745
3475,100097508986637,,,выключай их,0,выключить что ?,-0.284118
3477,100097508986637,,,выключай их,2,выключить ?,-0.332019
3483,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,2,я не могу позволить вам уйти .,0.304185
3482,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,1,"мне показалось , вы меня зовете .",0.258659
3484,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,3,мне уже легче .,0.219066
3486,100149747456986,,как вы здесь оказались ?,не надо меня тянуть !,5,"что "" не надо "" ? не надо меня .",0.054430
